In [1]:
using JuMP, Cbc, Clp

In [2]:
thermal_plants = [1,2,3,4]
months = [1,2,3]
inflows = [1,2,3,4]
c0 = 2.592
ρ = 0.96
ci = [10, 20, 40, 80]
cd = 500
m = 1000
vi = 2050
y = [150 0 0 0; 450 300 0 0; 771 213 771 213] # y[periodo,cenario]
vmax = 4100
qmax = 1500
ptmax = [100,150,200,250]

4-element Vector{Int64}:
 100
 150
 200
 250

In [3]:
master = Model(Clp.Optimizer)
# set_optimizer_attribute(master, "LogLevel",0)
@variables(master,begin
    0 <= v[months,inflows] <= vmax
    0 <= q[months,inflows] <= qmax
    s[months,inflows] >= 0
    pt[thermal_plants,months,inflows] >= 0
    d[months,inflows] >= 0
end)

delete(master, v[1,2])
delete(master, v[1,3])
delete(master, v[1,4])
delete(master, v[2,3])
delete(master, v[2,4])

delete(master, q[1,2])
delete(master, q[1,3])
delete(master, q[1,4])
delete(master, q[2,3])
delete(master, q[2,4])

delete(master, s[1,2])
delete(master, s[1,3])
delete(master, s[1,4])
delete(master, s[2,3])
delete(master, s[2,4])

delete(master, d[1,2])
delete(master, d[1,3])
delete(master, d[1,4])
delete(master, d[2,3])
delete(master, d[2,4])

for t in thermal_plants
    delete(master, pt[t,1,2])
    delete(master, pt[t,1,3])
    delete(master, pt[t,1,4])
    delete(master, pt[t,2,3])
    delete(master, pt[t,2,4])
end

@constraints(master,begin
    sum(pt[i,1,1] for i in thermal_plants)+ d[1,1] + ρ*q[1,1] == m
    v[1,1] + c0*(q[1,1]+s[1,1]) == vi + c0*y[1,1]
        
    sum(pt[i,2,1] for i in thermal_plants)+ d[2,1] + ρ*q[2,1] == m
    v[2,1] + c0*(q[2,1]+s[2,1]) - v[1,1] == c0*y[2,1]
        
    sum(pt[i,2,2] for i in thermal_plants)+ d[2,2] + ρ*q[2,2] == m
    v[2,2] + c0*(q[2,2]+s[2,2]) - v[1,1] == c0*y[2,2]
        
    sum(pt[i,3,1] for i in thermal_plants)+ d[3,1] + ρ*q[3,1] == m
    v[3,1] + c0*(q[3,1]+s[3,1]) - v[2,1] == c0*y[3,1]
        
    sum(pt[i,3,2] for i in thermal_plants)+ d[3,2] + ρ*q[3,2] == m
    v[3,2] + c0*(q[3,2]+s[3,2]) - v[2,1] == c0*y[3,2]
        
    sum(pt[i,3,3] for i in thermal_plants)+ d[3,3] + ρ*q[3,3] == m
    v[3,3] + c0*(q[3,3]+s[3,3]) - v[2,2] == c0*y[3,3]
        
    sum(pt[i,3,4] for i in thermal_plants)+ d[3,4] + ρ*q[3,4] == m
    v[3,4] + c0*(q[3,4]+s[3,4]) - v[2,2] == c0*y[3,4]
        
    [t in thermal_plants],pt[t,1,1] <= ptmax[t]
    [t in thermal_plants, s in 1:2],pt[t,2,s] <= ptmax[t]
    [t in thermal_plants, s in 1:4],pt[t,3,s] <= ptmax[t]


end)

@objective(master, Min, (sum(ci[i]*pt[i,1,1] for i in thermal_plants)+ cd*d[1,1])+
                        (1/2)*sum(sum(ci[i]*pt[i,2,s] for i in thermal_plants)+ cd*d[2,s] for s in 1:2)+ 
                        (1/4)*sum(sum(ci[i]*pt[i,3,s] for i in thermal_plants)+ cd*d[3,s] for s in 1:4))
optimize!(master)
# print(master)

Coin0506I Presolve 14 (-28) rows, 45 (-11) columns and 58 (-39) elements
Clp0006I 0  Obj 0 Primal inf 7000 (7)
Clp0006I 18  Obj 38008.696
Clp0000I Optimal - objective value 38008.696
Coin0511I After Postsolve, objective 38008.696, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 38008.6963 - 18 iterations time 0.002, Presolve 0.00


In [4]:
objective_value(master)

38008.6962962963

In [5]:
println("==========Volumes==========")
println(round(value(v[1,1]),digits=2))
println(round(value(v[2,1]),digits=2))
println(round(value(v[2,2]),digits=2))
println(round(value(v[3,1]),digits=2))
println(round(value(v[3,2]),digits=2))
println(round(value(v[3,3]),digits=2))
println(round(value(v[3,4]),digits=2))
println("==========Hydro plant turbined outflow==========")
println(round(value(q[1,1]),digits=2))
println(round(value(q[2,1]),digits=2))
println(round(value(q[2,2]),digits=2))
println(round(value(q[3,1]),digits=2))
println(round(value(q[3,2]),digits=2))
println(round(value(q[3,3]),digits=2))
println(round(value(q[3,4]),digits=2))
println("==========Hydro plant spillage==========")
println(round(value(s[1,1]),digits=2))
println(round(value(s[2,1]),digits=2))
println(round(value(s[2,2]),digits=2))
println(round(value(s[3,1]),digits=2))
println(round(value(s[3,2]),digits=2))
println(round(value(s[3,3]),digits=2))
println(round(value(s[3,4]),digits=2))
println("==========Load shedding==========")
println(round(value(d[1,1]),digits=2))
println(round(value(d[2,1]),digits=2))
println(round(value(d[2,2]),digits=2))
println(round(value(d[3,1]),digits=2))
println(round(value(d[3,2]),digits=2))
println(round(value(d[3,3]),digits=2))
println(round(value(d[3,4]),digits=2))
                
for t in thermal_plants
    println("==========Generation of thermal plant==========")
    println(round(value(pt[t,1,1]),digits=2))
    println(round(value(pt[t,2,1]),digits=2))
    println(round(value(pt[t,2,2]),digits=2))
    println(round(value(pt[t,3,1]),digits=2))
    println(round(value(pt[t,3,2]),digits=2))
    println(round(value(pt[t,3,3]),digits=2))
    println(round(value(pt[t,3,4]),digits=2))
end

==========Volumes==========
953.8
635.2
257.9
0.0
0.0
0.0
0.0
==========Hydro plant turbined outflow==========
572.92
572.92
568.48
1016.06
458.06
870.5
312.5
==========Hydro plant spillage==========
0.0
0.0
0.0
0.0
0.0
0.0
0.0
==========Load shedding==========
0.0
0.0
0.0
0.0
0.0
0.0
0.0
==========Generation of thermal plant==========
100.0
100.0
100.0
24.58
100.0
100.0
100.0
==========Generation of thermal plant==========
150.0
150.0
150.0
0.0
150.0
64.32
150.0
==========Generation of thermal plant==========
200.0
200.0
200.0
0.0
200.0
0.0
200.0
==========Generation of thermal plant==========
0.0
0.0
4.26
0.0
110.26
0.0
250.0
